In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO(r"C:\Users\tanji\OneDrive\Cambridge\2\Project\ShedSense\node\src\models\yolov10n_saved_model\yolov10n_full_integer_quant_edgetpu.tflite", task="detect")

cap = cv2.VideoCapture(r"C:\Users\tanji\OneDrive\Cambridge\2\Project\recordings\2025-02-26_14-38-06.mp4")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

det_video = cv2.VideoWriter(r"C:\Users\tanji\OneDrive\Cambridge\2\Project\recordings\Detections\02-26", cv2.VideoWriter_fourcc(*"mp4v"), 15, (width, height))

prev_frame_grey = None
wakeup_time_left = 0

while cap.isOpened():
    ret, frame =  cap.read()
    if not ret:
        break
    
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    if prev_frame_grey is None:
        prev_frame_grey = frame_grey   
    
    frame_difference = cv2.absdiff(prev_frame_grey, frame_grey)
    
    _, filter_frame = cv2.threshold(frame_difference, 30, 255, cv2.THRESH_BINARY)
    
    cv2.imshow("Filtered", filter_frame)
    
    
    pixels_changed = np.sum(filter_frame)/255
   
    
    if pixels_changed > 5000:
        wakeup_time_left = 1 # How many more frames does it take before it decides that an event is over
    
    if wakeup_time_left > 0:
        frame = cv2.resize(frame, (640, 640))
        
        wakeup_time_left -= 1
        
        prev_frame_grey = frame_grey
        
        results = model(frame)
        annotated_frame = results[0].plot()
        
        det_video.write(annotated_frame)
        
        cv2.imshow("Detected", annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

ModuleNotFoundError: No module named 'tensorflow'